In [39]:
#imports
import pandas as pd
from helper import pickle_store, pickle_restore
import dateutil
from adem import *
from collections import Counter 


#fetch all rawdata and save as pickle format for speed.
ocean_raw_df = pd.read_csv("r4.2/psychometric.csv") # 2629979 rows, 11 columns
# Select small data till 1 April
pickle_store("pickle/ocean_rawdf_file", ocean_raw_df) #raw means not indexed by Timeindex

# http://spi.unob.cz/papers/2017/2017-03.pdf

ocean_raw_df['risk_score']=ocean_raw_df.apply( lambda x: x.O*(0.35) + x.C*(0.21)+ x.E*(0.45)+ x.A*(0.25)+ x.N*(0.65)  , axis=1)

psychometric_risk_dic=ocean_raw_df.set_index('user_id').to_dict()['risk_score']
psychometric_risk_dic
pickle_store("pickle/psychometric_risk_file", psychometric_risk_dic)

In [ ]:
#get list of insiders from the answers:
insider_raw_df = pd.read_csv("r4.2/insiders.csv")
insider_raw_df = insider_raw_df[8:78]


In [ ]:
insider_lst = list(insider_raw_df['user'])
len(insider_lst)
pickle_store("pickle/insider_list_file", insider_lst)


#find what insiders are not in my input data set till april
eid_role_dic = pickle_restore("pickle/eid_role_map_file")
user_set = set(eid_role_dic.keys())
set_insiders = set(insider_lst)

#this loop tells me that all insiders are in my trimmed data set.
for usr in set_insiders:
    if usr not in user_set:
        print (usr)

In [40]:
#Now calculating Measurements

import os

feature_path = 'pickle/'

fi_path=[]
for subdir, dirs, files in os.walk(feature_path):
    # print ("subdir=",subdir)
    # print ("dir=",dirs)
    for file in dirs:
        fi_path.append(file)
    break

print("fi_path=", fi_path)

insider_lst = pickle_restore("pickle/insider_list_file")

fi_path.remove('f2')
fi_path.remove('f3')


# col_names = ['feature', 'ubp', 'pbp', 'cbp']




anom_users_across_feature=[]
for fi in fi_path:
    print('*********************************************')
    print(fi)

    for subfolder, folder, files in os.walk('pickle/'+fi):
        # print (files)
        fi_anom_set_lst_dic = pickle_restore('pickle/'+fi+'/'+fi+'_anom_set_file')
        
        print("ubp=", len(fi_anom_set_lst_dic['ubp']))
        print("pbp=", len(fi_anom_set_lst_dic['pbp']))
        print("cbp=", len(fi_anom_set_lst_dic['cbp']))
        print('------')
        print("set ubp=", len(set(fi_anom_set_lst_dic['ubp'])))
        print("set pbp=", len(set(fi_anom_set_lst_dic['pbp'])))
        print("set cbp=", len(set(fi_anom_set_lst_dic['cbp'])))

        anom_users_across_feature += list(set(fi_anom_set_lst_dic['ubp']))+ list(set(fi_anom_set_lst_dic['pbp'])) +list(set(fi_anom_set_lst_dic['cbp']))


reported_user_set = set(anom_users_across_feature)

psychometric_risk_dic = pickle_restore("pickle/psychometric_risk_file")

#to check what is risk score of real attackers in answers
tempdic= {}
for attacker in insider_lst:
    # print(attacker, psychometric_risk_dic[attacker])
    tempdic[attacker]=psychometric_risk_dic[attacker]


#find which insider attacker is not caught.
total_not_caught=0
for attacker in insider_lst:
    if attacker not in reported_user_set:
        total_not_caught+=1
        print (attacker, "not caught, risk score=", psychometric_risk_dic[attacker])
print ("total not caught=",total_not_caught)


fi_path= ['f6', 'f1', 'f5', 'f2', 'f8', 'f11', 'f7', 'f10', 'f9', 'f3', 'f4']
*********************************************
f6
ubp= 80
pbp= 1485
cbp= 1585
------
set ubp= 64
set pbp= 89
set cbp= 91
*********************************************
f1
ubp= 6
pbp= 13
cbp= 481
------
set ubp= 3
set pbp= 7
set cbp= 8
*********************************************
f5
ubp= 24
pbp= 20212
cbp= 21035
------
set ubp= 14
set pbp= 213
set cbp= 237
*********************************************
f8
ubp= 3
pbp= 0
cbp= 0
------
set ubp= 3
set pbp= 0
set cbp= 0
*********************************************
f11
ubp= 43
pbp= 17569
cbp= 18873
------
set ubp= 27
set pbp= 224
set cbp= 243
*********************************************
f7
ubp= 4
pbp= 1
cbp= 1
------
set ubp= 4
set pbp= 1
set cbp= 1
*********************************************
f10
ubp= 40
pbp= 11367
cbp= 11387
------
set ubp= 30
set pbp= 223
set cbp= 227
*********************************************
f9
ubp= 57
pbp= 14243
cbp= 11286
------
set ubp= 

In [41]:
#filter out naive users who are caught too frequently. 
from collections import Counter

len(anom_users_across_feature)
len(set(anom_users_across_feature))   

#check who is repeatedly reported max times
reporteduser_counter = Counter(anom_users_across_feature) 
l = list(reporteduser_counter.values())

#remove users more than count 15.
trimmed_reported_user_set = reported_user_set.copy()
for element in reporteduser_counter:
    # print (element, reporteduser_counter[element])
    if reporteduser_counter[element] <= 1:#reporteduser_counter[element] >= 20 and psychometric_risk_dic[element] < 60:
        if element in trimmed_reported_user_set:
            trimmed_reported_user_set.remove(element)


#find which insider attacker is not caught.
total_not_caught=0
for attacker in insider_lst:
    if attacker not in trimmed_reported_user_set:
        total_not_caught+=1
        print (attacker, "not caught, risk score=", psychometric_risk_dic[attacker])
print ("total not caught=",total_not_caught)

# print(len(trimmed_reported_user_set))


print(len(trimmed_reported_user_set))
print(len(reported_user_set))



JRG0207 not caught, risk score= 63.269999999999996
KLH0596 not caught, risk score= 50.2
MAR0955 not caught, risk score= 59.44
PPF0435 not caught, risk score= 63.42
WDD0366 not caught, risk score= 54.47
JLM0364 not caught, risk score= 53.92
total not caught= 6
350
374


In [42]:
#false positives: number of users in reported list but not in answers.
fp_count=0
for reported_usr in trimmed_reported_user_set:
    if reported_usr not in insider_lst:
        fp_count+=1
print ("FP=", fp_count)

#false negatives: number of users not in reported list but in answers.
fn_count=0
eid_role_dic = pickle_restore("pickle/eid_role_map_file")
user_set = set(eid_role_dic.keys())
not_reported_users = user_set - trimmed_reported_user_set

for attacker in not_reported_users:
    if attacker in insider_lst:
        fn_count+=1
print ("FN=", fn_count)


#true positives: number of users in reported list and in answers.
tp_count=0
for reported_usr in trimmed_reported_user_set:
    if reported_usr in insider_lst:
        tp_count+=1
print ("TP=", tp_count)


#true negative: number of users  not in reported list and not in answers.
tn_count=0
eid_role_dic = pickle_restore("pickle/eid_role_map_file")
user_set = set(eid_role_dic.keys())
not_reported_users = user_set - trimmed_reported_user_set
for usr in not_reported_users:
    if usr not in insider_lst:
        tn_count+=1
print ("TN=", tn_count)


FP= 286
FN= 6
TP= 64
TN= 644


In [5]:
eid_role_dic = pickle_restore("pickle/eid_role_map_file")
user_set = set(eid_role_dic.keys())
not_reported_users = user_set - trimmed_reported_user_set
len(not_reported_users)

25

In [6]:

def getfp(trimmed_reported_user_set, insider_lst):
    fp_count=0
    for reported_usr in trimmed_reported_user_set:
        if reported_usr not in insider_lst:
            fp_count+=1
    print ("FP=", fp_count)
    return fp_count

def getfn(user_set, trimmed_reported_user_set, insider_lst):
    fn_count=0
    eid_role_dic = pickle_restore("pickle/eid_role_map_file")
    user_set = set(eid_role_dic.keys())
    not_reported_users = user_set - trimmed_reported_user_set

    for attacker in not_reported_users:
        if attacker in insider_lst:
            fn_count+=1
    print ("FN=", fn_count)
    return fn_count

def gettp(trimmed_reported_user_set, insider_lst):
    tp_count=0
    for reported_usr in trimmed_reported_user_set:
        if reported_usr in insider_lst:
            tp_count+=1
    print ("TP=", tp_count)
    return tp_count


def gettn(user_set, trimmed_reported_user_set, insider_lst):
    tn_count=0
    eid_role_dic = pickle_restore("pickle/eid_role_map_file")
    user_set = set(eid_role_dic.keys())
    not_reported_users = user_set - trimmed_reported_user_set
    for usr in not_reported_users:
        if usr not in insider_lst:
            tn_count+=1
    print ("TN=", tn_count)
    return tn_count


In [7]:
#prep csv data for plots:
import csv 
col_names = ['feature', 'fp', 'fn', 'tp', 'tn', 'AR', 'DR', 'TPR', 'TNR', 'FPR', 'FNR']

rowlist=[]
for fi in fi_path:
    print(fi)
    anom_users_curr_feature = []
    for subfolder, folder, files in os.walk('pickle/'+fi):
        # print (files)
        fi_anom_set_lst_dic = pickle_restore('pickle/'+fi+'/'+fi+'_anom_set_file')
        
        # print("ubp=", len(fi_anom_set_lst_dic['ubp']))
        # print("pbp=", len(fi_anom_set_lst_dic['pbp']))
        # print("cbp=", len(fi_anom_set_lst_dic['cbp']))
        print('------')
        # print("set ubp=", len(set(fi_anom_set_lst_dic['ubp'])))
        # print("set pbp=", len(set(fi_anom_set_lst_dic['pbp'])))
        # print("set cbp=", len(set(fi_anom_set_lst_dic['cbp'])))

        anom_users_curr_feature += list(set(fi_anom_set_lst_dic['ubp']))+ list(set(fi_anom_set_lst_dic['pbp'])) +list(set(fi_anom_set_lst_dic['cbp']))

        cf_reported_user_set = set(anom_users_curr_feature)


        print ("list=", len(anom_users_curr_feature))
        print ("set= ", len(set(anom_users_curr_feature)))   

        #check who is repeatedly reported max times
        cf_reporteduser_counter = Counter(anom_users_curr_feature) 
        l = list(cf_reporteduser_counter.values())

        # #remove users more than count 15.
        # cf_trimmed_reported_user_set = cf_reported_user_set.copy()
        # for element in cf_reporteduser_counter:
        #     # print (element, reporteduser_counter[element])
        #     if cf_reporteduser_counter[element] <= 4:#reporteduser_counter[element] >= 20 and psychometric_risk_dic[element] < 60:
        #         if element in cf_trimmed_reported_user_set:
        #             cf_trimmed_reported_user_set.remove(element)

        # print("cf_trimmed_reported_user_set=", len(cf_trimmed_reported_user_set))
        print("cf_reported_user_set=", len(cf_reported_user_set))



        #find which insider attacker is not caught.
        total_not_caught=0
        for attacker in insider_lst:
            if attacker not in cf_reported_user_set:
                total_not_caught+=1
                # print (attacker, "not caught, risk score=", psychometric_risk_dic[attacker])
        print ("total not caught=",total_not_caught)

        fp = getfp(cf_reported_user_set, insider_lst)
        fn = getfn(user_set, cf_reported_user_set, insider_lst)
        tp = gettp(cf_reported_user_set, insider_lst)
        tn = gettn(user_set, cf_reported_user_set, insider_lst)

        # 'AR', 'DR', 'TPR', 'TNR', 'FPR', 'FNR']
        rowlist.append([fi, fp, fn, tp, tn, ((tn+tp)/(tn+tp+fn+fp)), ((tp)/(tp+fp)), ((tp)/(tp+fn)), ((tn)/(tn+fp)), ((fp)/(tn+fp)), ((fn)/(tp+fn))])

op_filename = "measure_stats.csv"
with open(op_filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(col_names)  
        
    # writing the data rows  
    csvwriter.writerows(rowlist) 


f6
------
list= 255
set=  85
cf_reported_user_set= 85
total not caught= 39
FP= 54
FN= 39
TP= 31
TN= 876
f1
------
list= 162
set=  54
cf_reported_user_set= 54
total not caught= 35
FP= 19
FN= 35
TP= 35
TN= 911
f5
------
list= 123
set=  41
cf_reported_user_set= 41
total not caught= 41
FP= 12
FN= 41
TP= 29
TN= 918
f8
------
list= 90
set=  30
cf_reported_user_set= 30
total not caught= 70
FP= 30
FN= 70
TP= 0
TN= 900
f11
------
list= 171
set=  57
cf_reported_user_set= 57
total not caught= 42
FP= 29
FN= 42
TP= 28
TN= 901
f7
------
list= 75
set=  25
cf_reported_user_set= 25
total not caught= 67
FP= 22
FN= 67
TP= 3
TN= 908
f10
------
list= 393
set=  131
cf_reported_user_set= 131
total not caught= 32
FP= 93
FN= 32
TP= 38
TN= 837
f9
------
list= 402
set=  134
cf_reported_user_set= 134
total not caught= 46
FP= 110
FN= 46
TP= 24
TN= 820
f4
------
list= 336
set=  112
cf_reported_user_set= 112
total not caught= 28
FP= 70
FN= 28
TP= 42
TN= 860
